In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="""Create an effective prompt template for getting flights""")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Create an effective prompt template for getting flights', additional_kwargs={}, response_metadata={}, id='77200e73-16dd-4bb2-b765-d112e605c080'),
              AIMessage(content='**Flight‑Search Prompt Template**\n\nUse this template whenever you want to query the Kiwi flight‑search tool.  \nReplace the bracketed placeholders with the user’s preferences.\n\n```json\n{\n  "cabinClass": "[M|W|C|F]",          // M=Economy, W=Economy Premium, C=Business, F=First\n  "curr": "[EUR|USD|GBP|JPY|CAD|AUD|NZD|CHF|…]",  // Currency code\n  "departureDate": "[dd/mm/yyyy]",    // Must be a future date\n  "departureDateFlexRange": [0–3],    // Days before/after the departure date\n  "flyFrom": "[City or Airport Code or Name]",\n  "flyTo": "[City or Airport Code or Name]",\n  "locale": "[en|de|fr|es|it|ru|…]",  // Language for city names & links\n  "passengers": {\n    "adults": [1–9],\n    "children": [0–9],\n    "infants": [0–9]\n  },\n  "returnDate": "[dd/mm/yyyy

In [7]:
print(response["messages"][-1].content)

**Flight‑Search Prompt Template**

Use this template whenever you want to query the Kiwi flight‑search tool.  
Replace the bracketed placeholders with the user’s preferences.

```json
{
  "cabinClass": "[M|W|C|F]",          // M=Economy, W=Economy Premium, C=Business, F=First
  "curr": "[EUR|USD|GBP|JPY|CAD|AUD|NZD|CHF|…]",  // Currency code
  "departureDate": "[dd/mm/yyyy]",    // Must be a future date
  "departureDateFlexRange": [0–3],    // Days before/after the departure date
  "flyFrom": "[City or Airport Code or Name]",
  "flyTo": "[City or Airport Code or Name]",
  "locale": "[en|de|fr|es|it|ru|…]",  // Language for city names & links
  "passengers": {
    "adults": [1–9],
    "children": [0–9],
    "infants": [0–9]
  },
  "returnDate": "[dd/mm/yyyy]",       // Optional – leave blank or null for one‑way
  "returnDateFlexRange": [0–3],       // Optional – leave blank or null for one‑way
  "sort": "[price|duration|quality|date]"   // Default is date
}
```

**Example (One‑Way)**  


In [8]:
prompt = """
{
  "cabinClass": "C",
  "curr": "USD",
  "departureDate": "04/11/2026",
  "departureDateFlexRange": 2,
  "flyFrom": "New York JFK",
  "flyTo": "Tokyo NRT",
  "locale": "en",
  "passengers": {"adults": 2, "children": 1, "infants": 0},
  "returnDate": "10/08/2026",
  "returnDateFlexRange": 1,
  "sort": "duration"
}
"""


response = await agent.ainvoke(
    {"messages": [HumanMessage(content=prompt)]},
    config
    )


pprint(response)

McpError: MCP error -32602: MCP error -32602: Invalid arguments for tool search-flight: [
  {
    "validation": "regex",
    "code": "invalid_string",
    "message": "date must be in dd/mm/yyyy format",
    "path": [
      "returnDate"
    ]
  }
]

In [ ]:
print(response["messages"][-1].content)